In [41]:
import numpy as np
from parse_rest.connection import register
from parse_rest.datatypes import Object
import key
from sklearn.linear_model import LogisticRegression

register(key.APP_ID, key.REST_API_KEY)

class DataChunk(Object):
    pass

In [27]:
allData = DataChunk.Query.all()
hcData = allData.filter(userId='acsalu')
coData = allData.filter(userId='copila')
jeanData = allData.filter(userId='jean')

In [28]:
featureName = ['accelerationMagnitudes', 'totalNumberOfDeletions', 'gyroMagnitudes', \
                'interTapDistances', 'tapDurations']
emotions = {'Happy': 0, 'Sad': 1, 'Neutral': 2}

In [10]:
print(allData[0].userId)

acsalu


In [29]:
print(coData)
print(allData)

[]
[<DataChunk:HrYUYIwIdh>, <DataChunk:lDuwFI0DSx>, <DataChunk:rDrbIcS9vD>]


In [66]:
labels = list(map(lambda x: emotions[x.emotion], allData))
[accMag, ttlNODel, gyro, intTapDist, tapDur] = \
    [[getattr(data, featureName[idx]) for data in allData] for idx in range(len(featureName))]
    
aveAccMag, stdAccMag = [np.mean(a) for a in accMag], [np.std(a) for a in accMag]
aveGyro, stdGyro = [np.mean(g) for g in gyro], [np.std(g) for g in gyro]
aveIntTapDist, stdIntTapDist = [np.mean(i) for i in intTapDist], [np.std(i) for i in intTapDist]
aveTapDur, stdTapDur = [np.mean(t) for t in tapDur], [np.std(t) for t in tapDur]

In [62]:
print(aveAccMag, stdAccMag)
print(aveGyro, stdGyro)
print(aveIntTapDist, stdIntTapDist)
print(aveTapDur, stdTapDur)

[0.023283081131560662, 0.023283081131560662, 0.029472699502537454] [0.016076728480291985, 0.016076728480291985, 0.027670591952730441]
[0.17171352788901648, 0.17171352788901648, 0.19508022618563575] [0.14149932138246304, 0.14149932138246304, 0.15513328893765371]
[0.40352022915612906, 0.40352022915612906, 0.94013914460425851] [0.27008035418111825, 0.27008035418111825, 3.2737306412931355]
[501.6062083304858, 501.6062083304858, 177.25180324568001] [573.99714874706399, 573.99714874706399, 358.6273725206388]


# Normalization

# Logistic Regression

In [67]:
features = [aveAccMag, stdAccMag, ttlNODel, aveGyro, stdGyro, aveIntTapDist, stdIntTapDist, aveTapDur, stdTapDur]
features = np.array(features).T
print(features.shape)
print(labels)

(3, 9)
[2, 0, 1]


In [69]:
model = LogisticRegression()
model.fit(features, labels)
print(model.score(features, labels))

0.666666666667
